In [1]:
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q peft
!pip install -q jsonargparse

In [2]:
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from pathlib import Path
from typing import Optional
from torch.utils.data import random_split
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
from torch import nn, optim
import torch
import os
from tqdm.notebook import tqdm
from torch.cuda.amp import GradScaler, autocast
import torch.nn.functional as F
from transformers import BitsAndBytesConfig
import numpy as np
from transformers import get_cosine_schedule_with_warmup

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# General Settings

In [3]:
llm_backbone = 'mistralai/Mistral-7B-v0.1'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_seq_length = 380
num_epochs = 1
micro_batch_size = 2
learning_rate = 0.00003
warmup_steps = 1000
weight_decay = 0.005
eval_steps = 200
logging_step = 50
accumulation_steps = 2

# Prepare data

In [4]:
%%writefile prepare_data.py

import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from pathlib import Path
from torch.utils.data import random_split
import torch
import os

def generate_prompt(example: dict) -> str:
    """Generates a standardized message to prompt the model with an instruction,
       optional input and a 'response' field."""

    if example["input"]:
        return (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:"
        )
    return (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{example['instruction']}\n\n### Response:"
    )


def prepare_sample(example, tokenizer, masked_inputs, ignore_index):
    example = example.to_dict()
    full_prompt = generate_prompt(example)
    full_prompt_and_response = full_prompt + example['response']
    encoded_full_prompt = tokenizer.encode(full_prompt)
    eos_id = tokenizer.eos_token
    encoded_full_prompt_and_response = torch.cat(
        [
            tokenizer.encode(full_prompt_and_response, return_tensors='pt').view(-1),
            tokenizer.encode(eos_id, return_tensors='pt', add_special_tokens=False).view(-1)
        ]
    )

    labels = encoded_full_prompt_and_response.clone()
    
    if masked_inputs:
        labels[:len(encoded_full_prompt)] = ignore_index
        
    return {
        **example,
        'input_ids': encoded_full_prompt_and_response,
        'input_ids_no_response': encoded_full_prompt,
        'labels': labels
    }

def prepare(
    test_size=0.1,
    destination_path=Path('data/dolly'),
    checkpoint_dir='mistralai/Mistral-7B-v0.1',
    seed=252
):
    destination_path.mkdir(parents=True, exist_ok=True)
    
    print('Loading tokenizer ...')
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
    
    data_file_url = 'https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl'
    data = pd.read_json(data_file_url, lines=True)
    data.columns = ['instruction', 'input', 'response', 'category']
    data = [example for _, example in data.iterrows()]
    
    train_set, test_set = random_split(
        data, 
        [1 - test_size, test_size], 
        torch.Generator().manual_seed(seed)
    )

    train_set, test_set = list(train_set), list(test_set)
    
    print(f'Train has {len(train_set)} samples')
    print(f'Test has {len(test_set)} samples')
    
    print('Processing train split ...')
    train_set = [
        prepare_sample(
            sample, 
            tokenizer=tokenizer,
            masked_inputs=False, 
            ignore_index=-1
        )
        for sample in tqdm(train_set)
    ]
    
    torch.save(train_set, destination_path / 'train_set.pt')
    
    print('Processing test split ...')
    test_set = [
        prepare_sample(
            sample, 
            tokenizer=tokenizer,
            masked_inputs=False, 
            ignore_index=-1
        )
        for sample in tqdm(test_set)
    ]
    
    torch.save(test_set, destination_path / 'test_set.pt')
    
if __name__ == "__main__":
    from jsonargparse import CLI

    CLI(prepare)

Writing prepare_data.py


In [5]:
!python prepare_data.py

Loading tokenizer ...
Train has 13510 samples
Test has 1501 samples
Processing train split ...
100%|████████████████████████████████████| 13510/13510 [00:21<00:00, 631.70it/s]
Processing test split ...
100%|██████████████████████████████████████| 1501/1501 [00:02<00:00, 643.79it/s]


# Load data

In [6]:
train_data = torch.load("/kaggle/working/data/dolly/train_set.pt")
val_data = torch.load('/kaggle/working/data/dolly/test_set.pt')

In [7]:
def longest_seq_length(data):
    lengths = [len(d['input_ids']) for d in data]
    longest_seq_length = max(lengths)
    longest_seq_idx = lengths.index(longest_seq_length)
    return longest_seq_length, longest_seq_idx

In [8]:
torch.manual_seed(289)
def get_batch(data, mode, train_idx, longest_seq_idx, max_seq_length, micro_batch_size):
    
    if mode == 'train':
        idx = train_idx
    else:
        idx = torch.randint(len(data), (micro_batch_size,))

    if longest_seq_idx is not None:
        idx[0] = longest_seq_idx

    input_ids = [data[i]['input_ids']for i in idx]
    labels = [data[i]['labels']for i in idx]

    max_len = max([len(s) for s in input_ids])
    
    def pad_right(x, pad_id):
        n = max_len - len(x)
        return torch.cat([x, torch.full((n,), pad_id)], dim=0)
    
    x = torch.stack([pad_right(x, pad_id=1) for x in input_ids])
    y = torch.stack([pad_right(y, pad_id=-1) for y in labels])
    
    if max_seq_length:
        x = x[:, :max_seq_length]
        y = y[:, :max_seq_length]
        
    return x.to(device), y.to(device)

input_ids, targets = get_batch(
    train_data, 
    train_idx=torch.tensor([0, 1]),
    mode='train', 
    longest_seq_idx=None, 
    max_seq_length=500, 
    micro_batch_size=4
)

# Tokenizizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained(llm_backbone)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    
tokenizer.decode(train_data[597]['input_ids'])

'<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nIs Leonidas from Sparta a real man? What was he famous for?\n\n### Response:Yes, Leonidas from Sparta was a real man. He was famous for his courage and leadership during the Battle of Thermopylae in 480 BC.</s>'

# Architecture

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.float16
        )
        
        self.backbone =  AutoModelForCausalLM.from_pretrained(
            llm_backbone,
            quantization_config=bnb_config,
            load_in_4bit=True,
        )
        
        self.peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM, 
            inference_mode=False,
            r=8,
            lora_alpha=16, 
            lora_dropout=0.05
        )
        
        self.backbone = get_peft_model(self.backbone, self.peft_config )
        self.backbone.print_trainable_parameters()
        
    def forward(self, input_ids, targets):
        logits = self.backbone(input_ids).logits
        logits = logits[..., :-1, :]
        targets = targets[..., 1:]
        
        B, T, C = logits.shape
        logits = logits.reshape(B*T, C)
        targets = targets.reshape(-1)
        
        loss = F.cross_entropy(logits, targets, ignore_index=-1)
        
        return logits, loss

# Fine-tuning

In [11]:
torch.manual_seed(279)
model = Net().to(device)

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


train_indices = []
shuffled_indices = torch.randperm(len(train_data))
for step in range(0, len(train_data)-micro_batch_size, micro_batch_size):
    indices = shuffled_indices[step:step+micro_batch_size]
    if len(indices) == micro_batch_size:
        train_indices.append(indices)
            
scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps, 
    num_training_steps=len(train_indices)*num_epochs
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.04703666202518836


In [12]:
torch.manual_seed(279)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    
    shuffled_indices = torch.randperm(len(train_data))
    train_indices = []
    for step in range(0, len(train_data)-micro_batch_size, micro_batch_size):
        indices = shuffled_indices[step:step+micro_batch_size]
        if len(indices) == micro_batch_size:
            train_indices.append(indices)
    
    train_indices = tqdm(train_indices, desc=f'Epoch: {epoch+1}')
    
    for batch_idx, batch in enumerate(train_indices):
        
        model.train()

        input_ids, targets = get_batch(
            train_data,
            train_idx=batch,
            mode='train',
            micro_batch_size=None,
            longest_seq_idx=None,
            max_seq_length=max_seq_length
        )

        _, loss = model(input_ids, targets)

        train_losses.append(loss.item())
        loss = loss / accumulation_steps  
        loss.backward()

        if (batch_idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        scheduler.step()

        if not batch_idx % logging_step:
            with torch.no_grad():
                model.eval()

                input_ids, targets = get_batch(
                    val_data,
                    train_idx=None,
                    mode='val',
                    micro_batch_size=micro_batch_size,
                    longest_seq_idx=None,
                    max_seq_length=max_seq_length
                )
                
                _, val_loss = model(input_ids, targets)
                val_losses.append(val_loss.item())

                print(
                f'Epoch: {epoch + 1}/{num_epochs}'
                f' | Batch: {batch_idx}/{len(train_indices)}'
                f' | Train Loss: {np.mean(train_losses)}'
                f' | Val loss: {np.mean(val_losses)}'
                )


        if not batch_idx % 100:
            with torch.no_grad():
                model.eval()
                text = """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nTell me about Viettel Networks (VTNet)?\n\n### Response:\n"""
                answer = tokenizer.decode(
                    model.backbone.generate(
                        **tokenizer(text, return_tensors='pt').to(device),
                        max_new_tokens=300,
                        pad_token_id=tokenizer.pad_token_id,
                    )[0]
                )
                print(answer)

Epoch: 1:   0%|          | 0/6754 [00:00<?, ?it/s]

Epoch: 1/1 | Batch: 0/6754 | Train Loss: 1.7164344787597656 | Val loss: 2.4670283794403076
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me about Viettel Networks (VTNet)?

### Response:

Viettel Networks (VTNet) is a Vietnamese telecommunications company that provides a range of services, including mobile, fixed-line, and internet services. The company was founded in 2001 and is headquartered in Hanoi, Vietnam.

VTNet is a subsidiary of Viettel Group, which is a state-owned enterprise that is majority-owned by the Vietnamese government. The company has a strong presence in Vietnam, with a network that covers over 90% of the country’s population.

VTNet offers a range of mobile services, including voice, data, and SMS services. The company also provides fixed-line services, including broadband and voice services. In addition, VTNet offers a range of internet services, including broadband, mobile data

In [13]:
# def chatbot():
#     question = input('Question:')
#     with torch.no_grad():
#         model.eval()
#         text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{question}\n\n### Response:"""
#         answer_ids =  model.backbone.generate(
#                 **tokenizer(text, return_tensors='pt').to(device),
#                 max_new_tokens=1000,
#                 pad_token_id=tokenizer.pad_token_id,
#         )[0]

#         start_idx = torch.where(answer_ids == 12107)[0] + 2

#         print('Answer:', tokenizer.decode(answer_ids[start_idx:-1]))

In [14]:
torch.save(model.state_dict(), 'mistral_7b_dolly.pt')